##1  PySpark - Instalando a biblioteca no google colab


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=8b1f8d169d9f0aad94306fc9e84e84bdcfed560a70437fdf251bfc729256c249
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
df=spark.sql('''select 'Sucesso total, estamos online!' as hello''')
df.show()

+--------------------+
|               hello|
+--------------------+
|Sucesso total, es...|
+--------------------+



In [5]:
#importando as bibliotecas do spark
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F #for more SQL functions
from functools import reduce

##2 Data Manipulations using Spark

In [6]:
df=spark.read.csv("/content/banklist.csv", sep=",", inferSchema=True, header=True)

print("df.count: ", df.count())
print("df.col ct: ", len(df.columns))
print("df.columns: ", df.columns)

df.count:  561
df.col ct:  6
df.columns:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


##3 Using SQL in PySpark

In [7]:
df.createOrReplaceTempView("banklist")
df_check=spark.sql('''select `Bank Name`, city, `Closing Date` from banklist''')
df_check.show()

+--------------------+------------------+------------+
|           Bank Name|              city|Closing Date|
+--------------------+------------------+------------+
|The First State Bank|     Barboursville|    3-Apr-20|
|  Ericson State Bank|           Ericson|   14-Feb-20|
|City National Ban...|            Newark|    1-Nov-19|
|       Resolute Bank|            Maumee|   25-Oct-19|
|Louisa Community ...|            Louisa|   25-Oct-19|
|The Enloe State Bank|            Cooper|   31-May-19|
|Washington Federa...|           Chicago|   15-Dec-17|
|The Farmers and M...|           Argonia|   13-Oct-17|
| Fayette County Bank|        Saint Elmo|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee|    5-May-17|
|      First NBC Bank|       New Orleans|   28-Apr-17|
|       Proficio Bank|Cottonwood Heights|    3-Mar-17|
|Seaway Bank and T...|           Chicago|   27-Jan-17|
|Harvest Community...|        Pennsville|   13-Jan-17|
|         Allied Bank|          Mulberry|   23-Sep-16|
|The Woodb

In [8]:
df.createOrReplaceTempView("banklist")
df_check=spark.sql('''select `Bank Name`, city, `Closing Date` from banklist''')
df_check.show(4, truncate=False)

+--------------------------------+-------------+------------+
|Bank Name                       |city         |Closing Date|
+--------------------------------+-------------+------------+
|The First State Bank            |Barboursville|3-Apr-20    |
|Ericson State Bank              |Ericson      |14-Feb-20   |
|City National Bank of New Jersey|Newark       |1-Nov-19    |
|Resolute Bank                   |Maumee       |25-Oct-19   |
+--------------------------------+-------------+------------+
only showing top 4 rows



##4 DataFrame Basic Operations

In [9]:
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                null|   null|null|31685.68449197861|                 null|        null|
| stddev|                null|   null|null|16446.65659309965|                 null|        null|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [10]:
df.describe("City", "ST").show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   null|null|
| stddev|   null|null|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



Count, Columns and Schema

In [11]:
print("total de Linhas: ", df.count())
print("total de Colunas: ", len(df.columns))
print("Colunas: ", df.columns)
print("Tipo do Dado: ", df.dtypes)
print("Schema: ", df.schema)

total de Linhas:  561
total de Colunas:  6
Colunas:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipo do Dado:  [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema:  StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


In [12]:
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



##5 Remove Duplicates

In [13]:
df=df.dropDuplicates()
print("df.count: ", df.count())
print("df.columns: ", df.columns)

df.count:  561
df.columns:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


##6 Select Specific Columns

In [14]:
df2=df.select(*["Bank Name", "City"])
df2.show()

+--------------------+----------------+
|           Bank Name|            City|
+--------------------+----------------+
| First Bank of Idaho|         Ketchum|
|Amcore Bank, Nati...|        Rockford|
|        Venture Bank|           Lacey|
|First State Bank ...|           Altus|
|Valley Capital Ba...|            Mesa|
|Michigan Heritage...|Farmington Hills|
|Columbia Savings ...|      Cincinnati|
|       Fidelity Bank|        Dearborn|
|The Park Avenue Bank|        Valdosta|
|Western Commercia...|  Woodland Hills|
|        Syringa Bank|           Boise|
|Republic Federal ...|           Miami|
|Westside Communit...|University Place|
|   First United Bank|           Crete|
|HarVest Bank of M...|    Gaithersburg|
|            BankEast|       Knoxville|
|    Polk County Bank|        Johnston|
|Colorado Capital ...|     Castle Rock|
|         Access Bank|        Champlin|
|Pacific National ...|   San Francisco|
+--------------------+----------------+
only showing top 20 rows



##7 Select Multiple Columns

In [15]:
col_l=list(set(df.columns) - {"CERT", "ST"})
df2 = df.select(*col_l)
df2.show()

+---------------------+--------------------+------------+----------------+
|Acquiring Institution|           Bank Name|Closing Date|            City|
+---------------------+--------------------+------------+----------------+
|      U.S. Bank, N.A.| First Bank of Idaho|   24-Apr-09|         Ketchum|
|          Harris N.A.|Amcore Bank, Nati...|   23-Apr-10|        Rockford|
| First-Citizens Ba...|        Venture Bank|   11-Sep-09|           Lacey|
|         Herring Bank|First State Bank ...|   31-Jul-09|           Altus|
| Enterprise Bank &...|Valley Capital Ba...|   11-Dec-09|            Mesa|
|       Level One Bank|Michigan Heritage...|   24-Apr-09|Farmington Hills|
| United Fidelity B...|Columbia Savings ...|   23-May-14|      Cincinnati|
| The Huntington Na...|       Fidelity Bank|   30-Mar-12|        Dearborn|
|   Bank of the Ozarks|The Park Avenue Bank|   29-Apr-11|        Valdosta|
| First California ...|Western Commercia...|    5-Nov-10|  Woodland Hills|
|         Sunwest Bank|  

##8 Rename Column

In [16]:
df2 = df \
  .withColumnRenamed("Bank Name", "bank_name") \
  .withColumnRenamed("Acquiring Institution", "acq_institution") \
  .withColumnRenamed("Closing Date", "closing_date") \
  .withColumnRenamed("ST", "state") \
  .withColumnRenamed("CERT", "cert")

df2.show()

+--------------------+----------------+-----+-----+--------------------+------------+
|           bank_name|            City|state| cert|     acq_institution|closing_date|
+--------------------+----------------+-----+-----+--------------------+------------+
| First Bank of Idaho|         Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|   MI|34369|      Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati|   OH|32284|United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn|   MI|33883|The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   GA|19797|  B

##9 Add Columns

In [19]:
df2=df.withColumn("state", col("ST"))
df2.show()

+--------------------+----------------+---+-----+---------------------+------------+-----+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|state|
+--------------------+----------------+---+-----+---------------------+------------+-----+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|   ID|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|   IL|
|        Venture Bank|           Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|   WA|
|First State Bank ...|           Altus| OK| 9873|         Herring Bank|   31-Jul-09|   OK|
|Valley Capital Ba...|            Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|   AZ|
|Michigan Heritage...|Farmington Hills| MI|34369|       Level One Bank|   24-Apr-09|   MI|
|Columbia Savings ...|      Cincinnati| OH|32284| United Fidelity B...|   23-May-14|   OH|
|       Fidelity Bank|        Dearborn| MI|33883| The Huntington Na...|   30-Mar-12|   MI|

##10 Add Constant Column

In [20]:
df2 = df.withColumn("country", lit("US"))
df2.show()

+--------------------+----------------+---+-----+---------------------+------------+-------+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+----------------+---+-----+---------------------+------------+-------+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|     US|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|     US|
|        Venture Bank|           Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|     US|
|First State Bank ...|           Altus| OK| 9873|         Herring Bank|   31-Jul-09|     US|
|Valley Capital Ba...|            Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|     US|
|Michigan Heritage...|Farmington Hills| MI|34369|       Level One Bank|   24-Apr-09|     US|
|Columbia Savings ...|      Cincinnati| OH|32284| United Fidelity B...|   23-May-14|     US|
|       Fidelity Bank|        Dearborn| MI|33883| The Huntington Na...

##11 Drop Column

In [21]:
df2 = df.drop("CERT")
df2.show()

+--------------------+----------------+---+---------------------+------------+
|           Bank Name|            City| ST|Acquiring Institution|Closing Date|
+--------------------+----------------+---+---------------------+------------+
| First Bank of Idaho|         Ketchum| ID|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford| IL|          Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey| WA| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus| OK|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa| AZ| Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills| MI|       Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati| OH| United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn| MI| The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta| GA|   Bank of the Ozarks|   29-Apr-11|
|Western Commercia...|  Woodland Hills| CA| First Ca

##12 Drop Multiple Column

In [22]:
df2 = df.drop(*["CERT", "ST"])
df2.show()

+--------------------+----------------+---------------------+------------+
|           Bank Name|            City|Acquiring Institution|Closing Date|
+--------------------+----------------+---------------------+------------+
| First Bank of Idaho|         Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa| Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|       Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati| United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn| The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   Bank of the Ozarks|   29-Apr-11|
|Western Commercia...|  Woodland Hills| First California ...|    5-Nov-10|
|        Syringa Bank|   

In [24]:
#modelo diferente
df2 = reduce(DataFrame.drop, ["CERT", "ST"], df)
df2.show()

+--------------------+----------------+---------------------+------------+
|           Bank Name|            City|Acquiring Institution|Closing Date|
+--------------------+----------------+---------------------+------------+
| First Bank of Idaho|         Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa| Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|       Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati| United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn| The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   Bank of the Ozarks|   29-Apr-11|
|Western Commercia...|  Woodland Hills| First California ...|    5-Nov-10|
|        Syringa Bank|   

##13 Filter Data

In [25]:
#Equals do Values
df2 = df.where(df["ST"]=="NE")
df2.show()

+-------------------+---------+---+-----+---------------------+------------+
|          Bank Name|     City| ST| CERT|Acquiring Institution|Closing Date|
+-------------------+---------+---+-----+---------------------+------------+
|       TierOne Bank|  Lincoln| NE|29341|   Great Western Bank|    4-Jun-10|
|Sherman County Bank|Loup City| NE| 5431|        Heritage Bank|   13-Feb-09|
|Mid City Bank, Inc.|    Omaha| NE|19397|         Premier Bank|    4-Nov-11|
| Ericson State Bank|  Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+-------------------+---------+---+-----+---------------------+------------+



In [27]:
#Between values
df3=df.where(df["CERT"].between("1000", "2000"))
df3.show()

+--------------------+-------------+---+----+---------------------+------------+
|           Bank Name|         City| ST|CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+---+----+---------------------+------------+
|The Farmers Bank ...|    Lynchburg| TN|1690| Clayton Bank and ...|   15-Jun-12|
| Fayette County Bank|   Saint Elmo| IL|1802| United Fidelity B...|   26-May-17|
|     Bank of Ephraim|      Ephraim| UT|1249|        Far West Bank|   25-Jun-04|
| Citizens State Bank|New Baltimore| MI|1006|          No Acquirer|   18-Dec-09|
|Barnes Banking Co...|    Kaysville| UT|1252|          No Acquirer|   15-Jan-10|
|Glasgow Savings Bank|      Glasgow| MO|1056| Regional Missouri...|   13-Jul-12|
|           Hume Bank|         Hume| MO|1971|        Security Bank|    7-Mar-08|
|     Mainstreet Bank|  Forest Lake| MN|1909|         Central Bank|   28-Aug-09|
|      Heartland Bank|      Leawood| KS|1361|         Metcalf Bank|   20-Jul-12|
+--------------------+------

In [28]:
df4 = df.where(df["ST"].isin("NE", "IL"))
df4.show()

+--------------------+------------+---+-----+---------------------+------------+
|           Bank Name|        City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+------------+---+-----+---------------------+------------+
|Amcore Bank, Nati...|    Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
|   First United Bank|       Crete| IL|20685| Old Plank Trail C...|   28-Sep-12|
|    Corus Bank, N.A.|     Chicago| IL|13693| MB Financial Bank...|   11-Sep-09|
|   First Choice Bank|      Geneva| IL|57212|  Inland Bank & Trust|   19-Aug-11|
|Highland Communit...|     Chicago| IL|20290| United Fidelity B...|   23-Jan-15|
|Second Federal Sa...|     Chicago| IL|27986| Hinsdale Bank & T...|   20-Jul-12|
|Corn Belt Bank & ...|  Pittsfield| IL|16500| The Carlinville N...|   13-Feb-09|
|Independent Banke...| Springfield| IL|26820| The Independent B...|   18-Dec-09|
|       Broadway Bank|     Chicago| IL|22853| MB Financial Bank...|   23-Apr-10|
|Citizens National...|      

In [29]:
#contagem de cada filtro
df2 = df.where(df["ST"]=="NE")

df3=df.where(df["CERT"].between("1000", "2000"))

df4 = df.where(df["ST"].isin("NE", "IL"))

print("df.count: ", df.count())
print("df2.count: ", df2.count())
print("df3.count: ", df3.count())
print("df4.count: ", df4.count())

df.count:  561
df2.count:  4
df3.count:  9
df4.count:  73


##14 Filter Data Using Logical Operator

In [30]:
df2 = df.where((df["ST"]=="NE")&(df["CITY"]=="Ericson"))
df2.show()

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



## 15 Replace Values in DataFrame

In [33]:
# Pre Replace
df.show()

# Post Replace
print("replace 7 in the above dataframe with 17 at all instances")
df.na.replace(7,17).show()

+--------------------+----------------+---+-----+---------------------+------------+
|           Bank Name|            City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+----------------+---+-----+---------------------+------------+
| First Bank of Idaho|         Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus| OK| 9873|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills| MI|34369|       Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati| OH|32284| United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn| MI|33883| The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta| GA|19797|   Bank of the O